In [33]:
import pandas as pd
import pyodbc

In [37]:
server = "localhost"
database = "cars_2021"
username = "xxxx"
password = "xxxx"

connection = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};SERVER="
    + server
    + ";DATABASE="
    + database
    + ";UID="
    + username
    + ";PWD="
    + password
)

def exec_query(query:str) -> pd.DataFrame:
    return pd.read_sql_query(query, connection)

In [69]:
model = "escalade"
trim = "esv"

model_df = exec_query(f"""select make.name, model.name, tr.name, tr.start_production_year, tr.id_car_trim
from car_make make
inner join car_model model
on make.id_car_make = model.id_car_make
inner join car_trim tr
on tr.id_car_model = model.id_car_model
where model.name like '%{model}%' and tr.name like '%{trim}%'
order by tr.start_production_year desc, tr.name asc""")

model_df

,name,name,name,start_production_year,id_car_trim
0,Cadillac,Escalade,6.2 V8 AT 4WD ESV (409 hp),2014,69907
1,Cadillac,Escalade,6.2 V8 AT 4WD ESV (426 hp),2014,26076


In [70]:
trim_id = 26076

model_spec_df = exec_query(f"""select spec.name, spec.id_parent, specval.value, specval.unit
from car_specification as spec
inner join car_specification_value specval
on spec.id_car_specification = specval.id_car_specification
where specval.id_car_trim = {trim_id}
order by spec.id_parent""")

model_spec_df

,name,id_parent,value,unit
0,Body type,1,Crossover,None
1,Number of seater,1,7,None
2,Length,1,5697,mm
3,Width,1,2045,mm
4,Height,1,1880,mm
5,Wheelbase,1,3302,mm
6,Front track,1,1745,mm
7,Rear track,1,1744,mm
8,Curb weight,1,2739,kg
9,Ground clearance,1,205,mm


In [44]:
spec_parents_df = exec_query("select id_car_specification, name from car_specification where id_parent is null")
spec_parents_df

,id_car_specification,name
0,1631,Accumulator battery
1,1,Bodywork
2,11,Engine
3,23,Gearbox and handling
4,1549,General information
5,31,Operating characteristics
6,1551,Security
7,53,Steering
8,40,Suspension and brakes
9,1550,Volume and weight


In [71]:
model_spec_df = pd.merge(model_spec_df, spec_parents_df, left_on="id_parent", right_on="id_car_specification", how="inner")
model_spec_df.rename(columns = {"name_y": "spec_class", "name_x": "spec_name"}, inplace = True)
model_spec_df = model_spec_df[["spec_name", "value", "unit", "spec_class"]]
model_spec_df

,spec_name,value,unit,spec_class
0,Body type,Crossover,None,Bodywork
1,Number of seater,7,None,Bodywork
2,Length,5697,mm,Bodywork
3,Width,2045,mm,Bodywork
4,Height,1880,mm,Bodywork
5,Wheelbase,3302,mm,Bodywork
6,Front track,1745,mm,Bodywork
7,Rear track,1744,mm,Bodywork
8,Curb weight,2739,kg,Bodywork
9,Ground clearance,205,mm,Bodywork
